In [10]:
import psycopg2
import csv
import sqlite3

In [11]:
HEXACO_FACETS = [
    'sincerity',
    'fairness',
    'greed_avoidance',
    'modesty',
    'fearfulness',
    'anxiety',
    'dependence',
    'sentimentality',
    'social_self_esteem',
    'social_boldness',
    'sociability',
    'liveliness',
    'forgiveness',
    'gentleness',
    'flexibility',
    'patience',
    'organization',
    'diligence',
    'perfectionism',
    'prudence',
    'aesthetic_appreciation',
    'inquisitiveness',
    'creativity',
    'unconventionality',
]

HEXACO_FACTORS = [
    'honesty',
    'emotionality',
    'extraversion',
    'agreeableness',
    'conscientiousness',
    'openness'
]

In [12]:
# CHANGE YOUR DB PARAMETERS HERE
dsn_database = "veljkodb"
dsn_hostname = "127.0.0.1"
dsn_port = "5432"
dsn_uid = "veljkopg"     
dsn_pwd = "pntdcn15"

In [13]:
def connect_to_db():
    global cursor
    global conn
    try:
        conn_string = "host="+dsn_hostname+" port="+dsn_port+" dbname="+dsn_database+" user="+dsn_uid+" password="+dsn_pwd
        print "Connecting to database\n  ->%s" % (conn_string)
        conn=psycopg2.connect(conn_string)
        cursor = conn.cursor()
        print "Connected!\n"
    except:
        print "Unable to connect to the database."
    return cursor

In [14]:
def compute_facets(answers):
    def r(i):
        return [0, 5, 4, 3, 2, 1][i]

    ans = list(map(int, answers.split('|')))
    ans = [0] + ans
    hexaco_scores = {
        'sincerity': (ans[6] + r(ans[30]) + ans[54]) / 3.0,
        'fairness': (r(ans[12]) + ans[36] + ans[60]) / 3.0,
        'greed_avoidance': (ans[18] + r(ans[42])) / 2.0,
        'modesty': (r(ans[24]) + r(ans[48])) / 2.0,
        'fearfulness': (ans[5] + ans[29] + r(ans[53])) / 3.0,
        'anxiety': (ans[11] + r(ans[35])) / 2.0,
        'dependence': (ans[17] + r(ans[41])) / 2.0,
        'sentimentality': (ans[23] + ans[47] + r(ans[59])) / 3.0,
        'social_self_esteem': (ans[4] + r(ans[28]) + r(ans[52])) / 3.0,
        'social_boldness': (r(ans[10]) + ans[34] + ans[58]) / 3.0,
        'sociability': (ans[16] + ans[40]) / 2.0,
        'liveliness': (ans[22] + r(ans[46])) / 2.0,
        'forgiveness': (ans[3] + ans[27]) / 2.0,
        'gentleness': (r(ans[9]) + ans[33] + ans[51]) / 3.0,
        'flexibility': (r(ans[15]) + ans[39] + r(ans[57])) / 3.0,
        'patience': (r(ans[21]) + ans[45]) / 2.0,
        'organization': (ans[2] + r(ans[26])) / 2.0,
        'diligence': (ans[8] + r(ans[32])) / 2.0,
        'perfectionism': (r(ans[14]) + ans[38] + ans[50]) / 3.0,
        'prudence': (r(ans[20]) + r(ans[44]) + r(ans[56])) / 3.0,
        'aesthetic_appreciation': (r(ans[1]) + ans[25]) / 2.0,
        'inquisitiveness': (ans[7] + r(ans[31])) / 2.0,
        'creativity': (ans[13] + ans[37] + r(ans[49])) / 3.0,
        'unconventionality': (r(ans[19]) + ans[43] + r(ans[55])) / 3.0,
    }
    return hexaco_scores

In [15]:
def compute_factors(facets):
    hexaco_factors = {
        'honesty' : (facets['sincerity'] + facets['fairness']+facets['greed_avoidance']+facets['modesty']) / 4.0,
        'emotionality' : (facets['fearfulness'] + facets['anxiety']+facets['dependence']+facets['sentimentality']) / 4.0,
        'extraversion' : (facets['social_self_esteem'] + facets['social_boldness']+facets['sociability']+facets['liveliness']) / 4.0,
        'agreeableness' : (facets['forgiveness'] + facets['gentleness']+facets['flexibility']+facets['patience']) / 4.0,
        'conscientiousness' : (facets['organization'] + facets['diligence']+facets['perfectionism']+facets['prudence']) / 4.0,
        'openness' : (facets['aesthetic_appreciation'] + facets['inquisitiveness']+facets['creativity']+facets['unconventionality']) / 4.0
    }
    return hexaco_factors

In [16]:
conn = sqlite3.connect('/home/mfrlin/cognitive_load_web/app.db')

cur = conn.cursor()
cur.execute('SELECT * FROM user')
rows = cur.fetchall()

users = []
for row in rows:
    if row[2] is None:
        continue
    n2_correct, n2_wrong, n2_possible = row[3].split(';')
    n3_correct, n3_wrong, n3_possible = row[6].split(';')
    user = {'client_id': row[1],
            'n2_correct': n2_correct,
            'n2_incorrect': n2_wrong,
            'n2_all_correct': n2_possible,
            'n2_time_start': row[4],
            'n3_correct': n3_correct,
            'n3_incorrect': n3_wrong,
            'n3_all_correct': n3_possible,
            'n3_time_start': row[5],
            }
    facets = compute_facets(row[2])
    factors = compute_factors(facets)
    user.update(facets)
    user.update(factors)
    users.append(user)

#with open('web_data.csv', 'w') as csvfile:
#    fieldnames = ['ident', 'n2_correct', 'n2_wrong', 'n2_possible', 'n2_time',
#                  'n3_correct', 'n3_wrong', 'n3_possible', 'n3_time'] + HEXACO_KEYS
#    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#    writer.writeheader()
#    for u in users:
#        writer.writerow(u)
#for u in users:
#    print u['client_id'], u['honesty']

In [17]:
connect_to_db()

for u in users:
    if cursor != None:
        cursor.execute("""INSERT INTO user_profile (client_id, n2_correct, n2_incorrect, n2_all_correct, n2_time_start, n3_correct, n3_incorrect, n3_all_correct, n3_time_start, sincerity, fairness, greed_avoidance, modesty, fearfulness, anxiety, dependence, sentimentality, social_self_esteem, social_boldness, sociability, liveliness, forgiveness, gentleness, flexibility, patience, organization, diligence, perfectionism, prudence, aesthetic_appreciation, inquisitiveness, creativity, unconventionality, honesty, emotionality, extraversion, agreeableness, conscientiousness, openness) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", (u['client_id'], u['n2_correct'], u['n2_incorrect'], u['n2_all_correct'],  u['n2_time_start'], u['n3_correct'], u['n3_incorrect'], u['n3_all_correct'], u['n3_time_start'], u['sincerity'], u['fairness'], u['greed_avoidance'], u['modesty'], u['fearfulness'], u['anxiety'], u['dependence'], u['sentimentality'], u['social_self_esteem'],  u['social_boldness'],  u['sociability'],  u['liveliness'],  u['forgiveness'],  u['gentleness'], u['flexibility'],  u['patience'],  u['organization'], u['diligence'], u['perfectionism'], u['prudence'], u['aesthetic_appreciation'], u['inquisitiveness'], u['creativity'], u['unconventionality'], u['honesty'], u['emotionality'], u['extraversion'], u['agreeableness'], u['conscientiousness'], u['openness']))
        print "executed", u['client_id']

Connecting to database
  ->host=127.0.0.1 port=5432 dbname=veljkodb user=veljkopg password=pntdcn15
Connected!



IntegrityError: duplicate key value violates unique constraint "user_profile_pkey"
DETAIL:  Key (client_id)=(iz2ps) already exists.


In [9]:
# Run this to commit the changes 
# Alternatively, uncomment "rollback" and comment "commit" in order to start again.

conn.commit()
#conn.rollback()
cursor.close()
conn.close()